# Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "pandas\..*" )

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
# import math
import os
import datetime
from functools import partial
pd.set_option('display.max_columns', None)

from matplotlib import rc
rc('text',usetex=True)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
rc('text.latex', preamble='\\usepackage{color} \\usepackage{amsmath} \\usepackage{amssymb}')

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
DS_name = 'hazelnut'
codes_path = os.getcwd()
results_path=os.path.join(codes_path,         'results')
# path_csv    = os.path.join(results_path,          'csv')
path_csv    = '../results_logits_with_auc_clipped/'

path_boxplots    = os.path.join(results_path, 'boxplot')

if os.path.exists(results_path):
    os.makedirs(path_boxplots,              exist_ok=True)


In [ ]:
num_epochs = 30000
num_explained_classes = 1
batch_size = 16
num_samples = 500
postfix_csv = f'{DS_name}_9'
exp_name = 'csv_exp_E6'

exp_no = 'E6'


In [ ]:
def get_file_modify_time(filepath=None):
    if os.path.exists(filepath):
        datestamp  = datetime.datetime.fromtimestamp(os.path.getmtime(filepath))
        print('File Exists :', filepath)
        print('File Date/Time :', datestamp)

In [ ]:
# csv_filename = f'{path_csv}/csv_exp_{postfix_csv}_BPT.csv'
csv_filename = f'{path_csv}/{exp_name}_testresults_{postfix_csv}_BPT_new_eval.csv'
print(csv_filename,' *** ',os.path.exists(csv_filename))

get_file_modify_time(filepath = csv_filename)


In [ ]:
df = pd.read_csv(csv_filename)
df.head(3)

In [ ]:
computed_method = np.unique(df.method)
num_methods = len(computed_method)
num_images  = len(np.unique(df.img_no))

In [ ]:
total_anom_types = np.unique(df.a_type)
total_num_images = 0
for anom_type in total_anom_types:
    images_ls = np.unique(df[df.a_type==anom_type]['img_no'])
    print(anom_type,images_ls)
    total_num_images+=len(images_ls)
print(f'Computing Results for images: \t{total_num_images}')

In [ ]:
auc_methods      = [
                    # 'aucI_pred','aucD_pred',\
                    'aucI_clipr' ,'aucD_clipr', \
                    'max_IoU','au_IoU',\
                    'time_exp']

## ANOVA ANALYSIS


In [ ]:
from scipy.stats import f_oneway
print('='*108)
print(f'| {"num_images":<20} | {"num_methods":<25} | {"Experiment":53} |')

print(f'| {f"{total_num_images}":<20} | {num_methods:<25} | {exp_no:53} |')

print('='*108)
print(f'| {"METRIC":<20} | {"SIGNIFICANCE":<25} | {"PVALUE":<53} |')
print('-'*108)
methods = sorted(df.method.unique())
for score in auc_methods:
    if score=='time_exp':
        continue
    pops = []
    for m1 in methods:
        pops.append(df[score][df.method==m1].values)
    
    anova = f_oneway(*pops)
    print(f"| {score:<20} | {'H1' if anova.pvalue<0.05 else 'H0':<25} | {anova.pvalue:<53} |")
print('-'*108)

In [ ]:
verbose = False
def get_bpt_heatmaps(num_samples=1000,verbose=False):
    pass
def get_aa_heatmaps(num_samples=1000,verbose=False):
    pass
def get_heatmaps_lime(num_samples=1000,verbose=False):
    pass

In [ ]:
verbose = False

methods = [
    ('BPT-100',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=100,verbose=verbose)),
    ('BPT-500',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=500,verbose=verbose)),
    ('BPT-1000',         'xkcd:light pink',      partial(get_bpt_heatmaps, num_samples=1000,verbose=verbose)),

    ('AA-100',          'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=100, verbose=verbose)),
    ('AA-500',          'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=500, verbose=verbose)),
    ('AA-1000',         'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=1000, verbose=verbose)),
    
    ('LIME-50',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=50, num_samples=50*5,verbose=verbose)),
    ('LIME-100',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=100, num_samples=100*5,verbose=verbose)),
    ('LIME-200',        'xkcd:bright lime',     partial(get_heatmaps_lime, num_segments=200, num_samples=200*5,verbose=verbose)),
]
for n,_,_ in methods:
    print(n)

In [ ]:
name_remap = {
    'Partition-100': 'PE-100',
    'Partition-500': 'PE-500',
    'Partition-1000': 'PE-1000',
    'BPT-100': '\\bf BPT-100',
    'BPT-500': '\\bf BPT-500',
    'BPT-1000': '\\bf BPT-1000',
    'aIDG': '$|$IDG$|$',
    'aGradExpl': '$|$GradExpl$|$',
}
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_mse': '$\\mathit{MSE}^+$',
    'aucD_mse': '$\\mathit{MSE}^-$',
    'time_exp': 'log(time)'
}
method_colors = {
    'BPT-100':         '#f2a7c0',
    'BPT-500':         '#db587f',
    'BPT-1000':        '#9d2f4d',
    'AA-100':   '#4ed1f9',
    'AA-500':   '#00b1f2',
    'AA-1000':  '#008fe8',
    'AA-5000':  '#006bd7',
    'AA-10000':  '#3245bd',
    'LIME-100':         '#c7e883',
    'LIME-500':        '#8fc543',
    'LIME-1000':        '#66912d',
    'LRP':        '#008080',
    'GradCAM':         'xkcd:camel',
    'aIDG':            '#fcd459',
    'aGradExpl':       '#dd7c2b',
    'ShapGradE':       '#800000',
}
method_symbol = {
    'BPT-100'   :       '$\\clubsuit$1',
    'BPT-500'   :       '$\\clubsuit$2',
    'BPT-1000'  :       '$\\clubsuit$3',
    'AA-100'    :       '$\\boxplus$1',
    'AA-500'    :       '$\\boxplus$2',
    'AA-1000'   :       '$\\boxplus$3',
    'AA-5000'   :       '$\\boxplus$4',
    'AA-10000'  :       '$\\boxplus$5',
    'LIME-100'   :       '\\textbf{0}1',
    'LIME-500'  :      '\\textbf{0}2',
    'LIME-1000'  :      '\\textbf{0}3',
    'LRP'       :      '\\textbf{$\\Pi$}',
    'GradCAM'   :      '$\\bigtriangledown$',
    'aIDG'      :      '$\\divideontimes$',
    'aGradExpl' :     '$\\bigtriangleup$',
     'ShapGradE':       '$\\square$',
}

method_to_shortname = {}
for i,m in enumerate(method_colors.keys()):
    method_to_shortname[m] = f'{method_symbol[m]}'

In [ ]:
len(df.img_no[df.a_type==df.a_type.unique()[0]])

In [ ]:
print(num_methods,computed_method)

In [ ]:
print(f'Total Images: {num_images}')

In [ ]:
print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

In [ ]:
auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_mse', 'aucD_mse','max_IoU','au_IoU','time_exp']

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_adj': '$\\mathit{AUC}^+-Adju$',
    'aucD_adj': '$\\mathit{AUC}^--Adju$',
    
    'aucI_r': '$\\mathit{AUC}^+-R$',
    'aucD_r': '$\\mathit{AUC}^--R$',

    'aucI_adj_r': '$\\mathit{AUC}^+-Adju-R$',
    'aucD_adj_r': '$\\mathit{AUC}^--Adju-R$',


    'aucI_clip': '$\\mathit{AUC}^+-Cl$',
    'aucI_clipr': '$\\mathit{AUC}^+-Cl-R$',

    'aucD_clip': '$\\mathit{AUC}^--Cl$',
    'aucD_clipr': '$\\mathit{AUC}^--Cl-R$',

# aucI_clip, aucI_clipr	aucD_clip aucD_clipr


    'time_exp': 'log(time)'
}

In [ ]:
auc_methods      = ['aucI_pred','aucI_clip' , 'aucI_adj' , 'aucI_r', 'aucI_adj_r', 'max_IoU','time_exp',\
                    'aucD_pred','aucD_clipr', 'aucD_adj' , 'aucD_r', 'aucD_adj_r','au_IoU',\
                    ]

auc_method_ascending = ['aucI_pred','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r''max_IoU', 'au_IoU' ]

In [ ]:
df[df['time_total'] == df['time_total'].max()]

In [ ]:
threshold = df['time_total'].quantile(0.90)
df_top_90 = df[df['time_total'] > threshold]
df_top_90

In [ ]:
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


# auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_adj', 'aucD_adj', 'aucI_r','aucD_r',\
#                     'aucI_adj_r','aucD_adj_r', 'max_IoU', 'au_IoU', 'time_exp']

auc_methods      = ['aucI_pred','aucI_clip' , 'aucI_adj' , 'aucI_r', 'aucI_adj_r', 'max_IoU','time_exp',\
                    'aucD_pred','aucD_clipr', 'aucD_adj' , 'aucD_r', 'aucD_adj_r','au_IoU',\
                    ]

auc_method_ascending = ['aucI_pred','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r''max_IoU', 'au_IoU' ]

# total_images = len(df.image.unique())
total_methods = len(df.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 


# print(f'Explained Class: {explained_class_final}')
# print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
# print('Background type:\t',background_type)

print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

fig, axes = plt.subplots(2, int(ttl_cols/2), figsize=(15, 7))  # (rows, cols)
for j, score in enumerate(auc_methods):
    row, col = divmod(j, int(ttl_cols/2))
    ax = axes[row, col]
    # df = df[~df[score].isna()] 
    # df = df
    ascending = score in auc_method_ascending
    means = df.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = df[ df.method==method ][score]
        values = values[~values.isna()]
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])
    values = df[score]

    ##########################################
    # if score=='aucI_pred': min_x,max_x=  0,1.0 #-np.quantile(values,q=0.1),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*45)
    # if score=='aucI_pred': min_x,max_x= 0.01, 10

    
    
    if score=='aucI_r' : min_x,max_x= 0,1
    
    if score=='aucI_adj_r' : min_x,max_x= 0,1.5
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400

    if score=='aucD_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    if score=='aucD_pred': min_x,max_x= 0,np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    
    if score=='max_IoU'  : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    if score=='au_IoU'   : min_x,max_x= -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*30)
    ##########################################
    # print(score,min_x, max_x)
    # if score=='max_IoU':
    #     print(min_x,max_x)
    ##########################################
    if score=='time_exp' :
        min_x, max_x = 0.011, 10
        for m in [1, 3]:
            for k in range(1,5):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([1, 3, 5 ])
        ax.set_xticklabels(['1', '3', '5'])
    ##########################################
    if score=='aucI_r' : 
        min_x,max_x= 0.6,1.1
        ax.set_xlim(min_x, max_x)
    # ax.set_xlim(min_x, max_x)
    ax.set_title(f"{method_remap[score]}", fontsize=15)

    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
# filename_box_plot = f'{results_boxplots}/results_table_IoU_{len(np.unique(df.image))}_{len(methods)}_{background_type}'

plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)


################################
from matplotlib.patches import Patch

last_ax = axes[-1, -1]  # bottom-right subplot

# Unique method list in the order you want
lm = list(df.method.unique())

# Create legend handles with real name + symbol
legend_handles = []
for m in lm:
    shortname = method_to_shortname.get(m, m)
    fullname = name_remap.get(m, m)
    label = f'{shortname}: {fullname}'
    patch = Patch(
        facecolor=method_colors[m],
        edgecolor='blue' if m.startswith('BPT') else 'black',
        linewidth=1.5 if m.startswith('BPT') else 0.5,
        label=label
    )
    legend_handles.append(patch)
# for ti, tl in zip('XAI Methods','NONE'),(total_methods,['NONE']):
#     summary_patch = Patch(facecolor='white', edgecolor='white', label=[ti,tl])
#     legend_handles.append(summary_patch)

# Add legend to bottom-right subplot (split in 2 columns)
last_ax.legend(
    handles=legend_handles,
    ncol=1,
    loc='center left',
    # bbox_to_anchor=(1.05, 0.5),
    fontsize=7,
    frameon=False
)
plt.suptitle(f'[Experiment:{exp_no}_AD ] --- [total_methods : {total_methods}]\
             --- [Total Time: {str(datetime.timedelta(seconds=np.sum(df.time_total)))}]', fontsize=16)
plt.show()
# print('file saved at :', filename_box_plot)

## FOR PAPER

In [ ]:
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',

    'aucI_clipr': '$\\mathit{AUC}^+$',
    'aucD_clipr': '$\\mathit{AUC}^-$',

    'time_exp': 'log(time)'
}

arrow_up = '$\\Uparrow$'
arrow_down = '$\\Downarrow$'
arrow = {

    'max_IoU': arrow_up,
    'au_IoU': arrow_up,
    'aucI_clipr': arrow_up,
    
    'aucD_clipr': arrow_down,
    'time_exp': arrow_down,
}

In [ ]:
def box_plot(ax, data, label, position, edge_color, line_width, fill_color):
    bp = ax.boxplot(data,
                    # labels=[label],
                    tick_labels=[label],
                    positions=[position], vert=False, 
                    widths=0.5, whis=5, patch_artist=True, flierprops={'markersize': 1, 'markerfacecolor':'magenta', 'markeredgecolor':'magenta'}) # 
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=edge_color, linewidth=line_width)

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color)       
        
    return bp


auc_methods      = [
                    # 'aucI_pred','aucD_pred',\
                    'aucI_clipr' ,'aucD_clipr', \
                    'max_IoU','au_IoU',\
                    'time_exp']

auc_method_ascending = ['aucI_clipr','aucI_clip','aucI_adj','aucI_r', 'aucI_adj_r','max_IoU', 'au_IoU' ]

# total_images = len(df.image.unique())
total_methods = len(df.method.unique())
ttl_cols = len(auc_methods) if len(auc_methods)%2==0 else len(auc_methods)+1 

# print(f'Explained Class: {explained_class_final}')
# print(f'Total Images: {total_images}')
print(f'XAI Methods: {total_methods}')
# print('Background type:\t',background_type)

print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

fig, axes = plt.subplots(1, len(auc_methods), figsize=(12, 2.5))  # (rows, cols)
for j, score in enumerate(auc_methods):
    # row, col = divmod(j, int(ttl_cols/2))
    ax = axes[j]
    # df = df[~df[score].isna()] 
    # df = df
    ascending = score in auc_method_ascending
    means = df.groupby('method')[[score]].median() \
              .sort_values(score, ascending=ascending).index
    log_scale = (score=='time_exp')
    ax.set_xscale('log' if log_scale else 'linear')
    for i, method in enumerate(means):
        text_yincrement = -0.3
        values = df[ df.method==method ][score]
        values = values[~values.isna()]
        name = name_remap[method] if method in name_remap else method
        color = method_colors[method]
        q75 = np.quantile(values, q=0.75)
        q100 = np.quantile(values, q=1.0)
        if not log_scale and 'IoU' not in score and q100>1.0:
            q100, text_yincrement = 0.82, 0.15
        val_arr = [[v for v in values]]
        box_plot(ax, val_arr, '', i, 
                 'blue' if method.startswith('BPT') else 'black', 
                 1.5 if method.startswith('BPT') else 0.5,
                 color)
        # print(method,q100,score,np.max(values))
        if score in ['aucI_pred', 'aucD_pred']:
            ax.text(min(q100*1.3, 30) if log_scale else q75+0.5, i+text_yincrement, method_to_shortname[method])
        elif score in ['aucI_mse', 'aucD_mse']:
            ax.text(min(q100*1.3, 30) if log_scale else q100+60, i+text_yincrement, method_to_shortname[method])
        else:
            ax.text(min(q100*1.3, 30) if log_scale else q100+0.05, i+text_yincrement, method_to_shortname[method])
    values = df[score]

    ##########################################
    # if score=='aucI_pred': min_x,max_x=  0,1.0 #-np.quantile(values,q=0.1),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*45)
    # if score=='aucI_pred': min_x,max_x= 0.01, 10

    if score=='aucI_clipr' : min_x,max_x= 0.65,1.1
    if score=='aucD_clipr' : min_x,max_x= -0.05,0.4
    # if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400
    # if score=='aucI_mse' : min_x,max_x= -np.quantile(values,q=0.01),400

    # if score=='aucD_pred': min_x,max_x= 0,np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    print(f'score {score}')
    if score=='max_IoU'  : min_x,max_x= 0,1.15 # -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*20)
    if score=='au_IoU'   : min_x,max_x= -0.05,0.60 # -np.quantile(values,q=0.01),np.quantile(values,q=1)+(np.quantile(values,q=0)/np.quantile(values,q=1)*30)
    ##########################################
    # print(score,min_x, max_x)
    # if score=='max_IoU':
    #     print(min_x,max_x)
    ##########################################
    if score=='time_exp' :
        min_x, max_x = 0.5, 15
        for m in [1, 3]:
            for k in range(1,5):
                ax.axvline(m*k, lw=0.5, color='grey', alpha=1/k)
        ax.set_xticks([1, 3, 5 ])
        ax.set_xticklabels(['1', '3', '5'])
    ##########################################
    if score=='aucI_r' : 
        min_x,max_x= 0,1.5
        ax.set_xlim(min_x, max_x)
    ax.set_xlim(min_x, max_x)
    ax.set_title(f"{arrow[score]} {method_remap[score]}", fontsize=15)

    ax.set_ylim(-0.7, i+0.8)
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.10)
# filename_box_plot = f'{results_boxplots}/results_table_IoU_{len(np.unique(df.image))}_{len(methods)}_{background_type}'

plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.pdf', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.svg', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.savefig(f'{path_boxplots}/results_{exp_no}_table_XAD.png', dpi=150, transparent=True, bbox_inches='tight', pad_inches=0.02)
print(f'FILE SAVED: {path_boxplots}/results_{exp_no}_table_XAD.pdf')
# plt.suptitle(f'[Experiment:{exp_no}_{params.pretrained_model_type} ] --- [Total Images : {total_images}] --- [total_methods : {total_methods}]\
#              --- [Total Time: {str(datetime.timedelta(seconds=np.sum(df.time_total)))}]', fontsize=16)
plt.show()
# print('file saved at :', filename_box_plot)

# END